### Import stnl dataset

In [1]:
import pandas as pd
import numpy as np

stnl1 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/stnl_activity_subset_1.csv")
stnl2 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/stnl_activity_subset_2.csv")
stnl3 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/stnl_activity_subset_3.csv")
stnl4 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/stnl_activity_subset_4.csv")

stnl = pd.concat([stnl1, stnl2, stnl3, stnl4], ignore_index=True)

/var/folders/jl/kkh608tn0k194f6cv0bwbqp80000gn/T/ipykernel_84505/3461626665.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  stnl1 = pd.read_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/ogData/stnl_activity_subset_1.csv")


# Clean process
- Sort by date occur
- Run through new function *(filter_sth)*
- Run through cleaning process *(transform_data_AC)* to find 

In [2]:
stnl['Date'] = pd.to_datetime(stnl['ACTIVITY_DATETIME'])
stnl_date_sorted = stnl.sort_values(by=['CUSTOMER_KEY','Date'])

### Function for creating multiple lists

In [3]:
def filter_sth(df):
    # Convert 'Date' to datetime and extract year and month
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.dropna(subset=['Date'])
    df['year'] = df['Date'].dt.year.astype(int)
    df['month'] = df['Date'].dt.month.astype(int)

    # Initialize lists to store data for years with and without "primary_purchase_sth"
    sth_list = []
    non_sth_list = []

    # Iterate through each unique customer
    for customer_id in df['CUSTOMER_KEY'].unique():
        customer_data = df[df['CUSTOMER_KEY'] == customer_id]
        years = customer_data['year'].unique()
        
        # Iterate through each year for the current customer
        for year in years:
        
            # Define the date range for June 1 to May 30 of the following year
            start_date = pd.Timestamp(year=year, month=6, day=1)
            end_date = pd.Timestamp(year=year + 1, month=5, day=30)
            
            # Filter data within the specified date range
            year_span_data = customer_data[(customer_data['Date'] >= start_date) & (customer_data['Date'] <= end_date)]
            
            # Check for "primary_purchase_sth" in activity codes
            if (year_span_data['ACTIVITY_CODE'] == "primary_purchase_sth").any():
                sth_list.append(year_span_data)
            else:
                non_sth_list.append(year_span_data)

    # Concatenate the results for years with and without the purchase activity
    sth_df = pd.concat(sth_list, ignore_index=True) if sth_list else pd.DataFrame()
    non_sth_df = pd.concat(non_sth_list, ignore_index=True) if non_sth_list else pd.DataFrame()

    return sth_df, non_sth_df

# Usage:
#sth_df, non_sth_df = filter_purchase_activity(stnl_filtered)

In [4]:
stnl_with_sth, stnl_without_sth = filter_sth(stnl_date_sorted)

/var/folders/jl/kkh608tn0k194f6cv0bwbqp80000gn/T/ipykernel_84505/2492342828.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['Date'].dt.year.astype(int)
/var/folders/jl/kkh608tn0k194f6cv0bwbqp80000gn/T/ipykernel_84505/2492342828.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['Date'].dt.month.astype(int)


### Function for cleaning process:
Gets count of each activity code for each customer in a data frame for selected activities
- LOB Seahawks 
- LOB fgi
- email_click
- email unsub
- form submission
- forward received/send
- merch purchase
- page_view
- primary purchase club/ga/single/sth/suite
- secondary purchase/sale
- survey_submission

In [5]:
def transform_data_AC(df):
    # Create a copy to manipulate
    df_copy = df.copy()

    # Make dummy variables for LOB only counting non NaNs
    df_copy['LOB_SEAHAWKS'] = df_copy['LINE_OF_BUSINESS'] == "seahawks"
    df_copy['LOB_FGI'] = df_copy['LINE_OF_BUSINESS'] == "fgi"
    
    # Drop unneeded columns
    df_dropped = df_copy.drop(columns=['CUSTOMER_ACTIVITY_KEY', 'ACTIVITY_SOURCE', 'ACTIVITY_TYPE', 'ACTIVITY_DATETIME', 
                                       'LINE_OF_BUSINESS','IS_ENGAGEMENT_ACTIVITY','Date'])

    # Store Campaign Lost if applicable
    lost = False
    if "CAMPAIGN_LOST" in df_dropped.columns:
        lost = True
        lost_column = df_dropped[["CUSTOMER_KEY", "CAMPAIGN_LOST"]].drop_duplicates(subset=["CUSTOMER_KEY"], keep="first")
        df_dropped = df_dropped.drop(columns=["CAMPAIGN_LOST"])
    
    # Choose activities to remove and filter out
    #bad_Activities = ["email_send", "email_open", "email_sub", "email_incoming_from_customer", "email_outgoing_to_customer", 
    #                  "initial_creation", "forward_receipt", "phonecall_incoming_from_customer", "phonecall_outgoing_to_customer",
    #                 "text_incoming_from_customer", "text_outgoing_to_customer"]
    #df_filtered = df_dropped[~df_dropped['ACTIVITY_CODE'].isin(bad_Activities)]
    
    # Create dummy variables for ACTIVITY_CODE into 1's and 0's
    df_dummy_code = pd.get_dummies(df_dropped, columns=['ACTIVITY_CODE'], drop_first=True)
    df_dummy_code = df_dummy_code.astype(int)
    
    # Group by CUSTOMER_KEY and sum unique activity codes
    df_grouped = df_dummy_code.groupby('CUSTOMER_KEY').sum()

    # Divide the summed values by the Row_Count for each customer to standardize
    row_counts = df_dummy_code.groupby('CUSTOMER_KEY').size()
    df_grouped['Totals'] = row_counts

    # Add Campaign lost if was stored
    if lost == True:
        df_grouped = df_grouped.merge(lost_column, on="CUSTOMER_KEY", how="left")

    # relable rows to make an ID number
    df_grouped['ID'] = range(len(df_grouped))

    
    return df_grouped

In [7]:
clean_stnl_with_sth = transform_data_AC(stnl_with_sth)

### Function for standardizing
- Find percentages by dividing by the total number of activities per person 
- Rename variables

In [8]:
def standardize_Activity_Codes(df):
    df_copy = df.copy() 
    activity_cols = df_copy.filter(like="ACTIVITY_CODE").columns

    # Standardize by dividing by 'Totals' 
    df_copy[activity_cols] = df_copy[activity_cols].div(df_copy['Totals'], axis=0)

    # Rename columns to remove 'ACTIVITY_CODE_'
    df_copy.rename(columns={col: col.replace("ACTIVITY_CODE_", "") for col in activity_cols}, inplace=True)

    return df_copy

### Function for choosing important activity codes(purchases)

In [15]:
def purchases_clean(df, lost):
    columns_to_select = ["primary_purchase_club", "primary_purchase_ga",
                                   "primary_purchase_single","primary_purchase_sth",
                                   "primary_purchase_suite", "secondary_purchase",
                                   "secondary_sale", "merch_purchase", "percent_open", "percent_click", 
                                   "Totals"]
    existing_columns = [col for col in columns_to_select if col in df.columns]
    df_purchases = df[existing_columns]
 
    return df_purchases

### Create new variables and use functions

In [16]:
clean_stnl_with_sth["percent_open"] = clean_stnl_with_sth["ACTIVITY_CODE_email_open"]/clean_stnl_with_sth["ACTIVITY_CODE_email_send"]
clean_stnl_with_sth["percent_click"] = clean_stnl_with_sth["ACTIVITY_CODE_email_click"]/clean_stnl_with_sth["ACTIVITY_CODE_email_open"]

clean_stnl_with_sth_standard = standardize_Activity_Codes(clean_stnl_with_sth)
cleaner_stnl_with_sth = purchases_clean(clean_stnl_with_sth_standard, 0)

#stnl_purchases_clean.to_csv("/Users/dan/calpoly/BusinessAnalytics/GSB503HAWKS/data/cleanDataPurchase/stnl_purchases_clean.csv", index=False)
cleaner_stnl_with_sth

,primary_purchase_club,primary_purchase_ga,primary_purchase_single,primary_purchase_sth,secondary_purchase,secondary_sale,merch_purchase,percent_open,percent_click,Totals
CUSTOMER_KEY,,,,,,,,,,
-9222983717532481599,0.000000,0.0,0.007194,0.064748,0.007194,0.057554,0.0,0.425000,0.058824,139
-9044857705265524453,0.002597,0.0,0.000000,0.005195,0.000000,0.005195,0.0,0.025424,1.000000,385
-8963114232538807937,0.000000,0.0,0.000000,0.011173,0.000000,0.022346,0.0,0.138462,0.666667,179
-8932276230323054381,0.000000,0.0,0.000772,0.004633,0.000000,0.000772,0.0,0.267459,0.172222,1295
-8799106693862001648,0.000000,0.0,0.000000,0.001773,0.000000,0.001773,0.0,0.232143,0.128205,564
...,...,...,...,...,...,...,...,...,...,...
9104837920567636064,0.005882,0.0,0.000000,0.008824,0.000000,0.002941,0.0,0.225806,0.428571,340
9153609583084292144,0.000000,0.0,0.001167,0.003501,0.000000,0.000000,0.0,0.121359,1.760000,857
9175740394355393716,0.000000,0.0,0.000763,0.002288,0.000763,0.000763,0.0,0.612205,0.041801,1311
